In [26]:
import pandas as pd


In [27]:
df = pd.read_csv("../data/raw/energy/CC_LCL-FullData.csv")


In [5]:
df.head(5)

,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
0,MAC000002,Std,2012-10-12 00:30:00.0000000,0
1,MAC000002,Std,2012-10-12 01:00:00.0000000,0
2,MAC000002,Std,2012-10-12 01:30:00.0000000,0
3,MAC000002,Std,2012-10-12 02:00:00.0000000,0
4,MAC000002,Std,2012-10-12 02:30:00.0000000,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167932474 entries, 0 to 167932473
Data columns (total 4 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   LCLid                    object
 1   stdorToU                 object
 2   DateTime                 object
 3   KWH/hh (per half hour)   object
dtypes: object(4)
memory usage: 5.0+ GB


In [7]:
print(df.dtypes)
print(df.columns)


LCLid                      object
stdorToU                   object
DateTime                   object
KWH/hh (per half hour)     object
dtype: object
Index(['LCLid', 'stdorToU', 'DateTime', 'KWH/hh (per half hour) '], dtype='object')


In [8]:
df.columns = df.columns.str.strip()
print(df.columns)


Index(['LCLid', 'stdorToU', 'DateTime', 'KWH/hh (per half hour)'], dtype='object')


In [9]:
df["DateTime"] = pd.to_datetime(df["DateTime"])
print(df["DateTime"].dtype)

datetime64[ns]


In [10]:
df["KWH/hh (per half hour)"] = pd.to_numeric(df["KWH/hh (per half hour)"], errors="coerce")


In [11]:
print(df["KWH/hh (per half hour)"].dtype)
print(df["KWH/hh (per half hour)"].head())

float64
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: KWH/hh (per half hour), dtype: float64


In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167932474 entries, 0 to 167932473
Data columns (total 4 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   LCLid                   object        
 1   stdorToU                object        
 2   DateTime                datetime64[ns]
 3   KWH/hh (per half hour)  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 5.0+ GB


In [13]:
df.isna().sum()

LCLid                        0
stdorToU                     0
DateTime                     0
KWH/hh (per half hour)    5560
dtype: int64

In [14]:
df["KWH/hh (per half hour)"].describe()

count    1.679269e+08
mean     2.117630e-01
std      2.972592e-01
min      0.000000e+00
25%      5.800000e-02
50%      1.170000e-01
75%      2.390000e-01
max      1.076100e+01
Name: KWH/hh (per half hour), dtype: float64

In [15]:
for col in ["LCLid", "stdorToU"]:
    print(f"\n{col} unique values:")
    print(df[col].value_counts())



LCLid unique values:
LCLid
MAC000145    39752
MAC000147    39752
MAC000150    39747
MAC000152    39746
MAC000149    39745
             ...  
MAC005556        1
MAC001150        1
MAC005560        1
MAC005563        1
MAC005559        1
Name: count, Length: 5566, dtype: int64

stdorToU unique values:
stdorToU
Std    134148703
ToU     33783771
Name: count, dtype: int64


In [22]:
df[df["LCLid"] == "MAC000145"].head(1)

,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
4354648,MAC000145,Std,2011-11-23 09:00:00,0.355


In [23]:
df[df["LCLid"] == "MAC000147"].head(1)


,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
135007441,MAC000147,ToU,2011-11-23 10:00:00,0.15


In [24]:
df[df["LCLid"] == "MAC000150"].head(1)


,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
4473890,MAC000150,Std,2011-11-23 10:00:00,0.301


In [25]:
df[df["LCLid"] == "MAC000152"].head(1)


,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
4553375,MAC000152,Std,2011-11-23 12:00:00,0.241


In [39]:
target_lcl_id = "MAC000145"

df = df.rename(columns={"KWH/hh (per half hour) ": "KWH/hh"})
df["DateTime"] = pd.to_datetime(df["DateTime"])
df["KWH/hh"] = pd.to_numeric(df["KWH/hh"], errors="coerce")
df_filtered = df[df["LCLid"] == target_lcl_id].copy()
df_filtered = df_filtered.drop_duplicates()
df_filtered = df_filtered[["DateTime", "KWH/hh"]]
df_filtered.to_csv(f"../data/interim/energy/{target_lcl_id}.csv", index=False)

In [47]:
df_interpolated = pd.read_csv("../data/processed/energy/MAC000145_linear_interpolated.csv")


In [50]:
df_interpolated.isna().sum()

DateTime    0
KWH/hh      0
dtype: int64

In [48]:
df_interpolated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39727 entries, 0 to 39726
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DateTime  39727 non-null  object 
 1   KWH/hh    39727 non-null  float64
dtypes: float64(1), object(1)
memory usage: 620.9+ KB


In [49]:
from datetime import timedelta

df_interpolated["DateTime"] = pd.to_datetime(df_interpolated["DateTime"])
time_diffs = df_interpolated["DateTime"].diff().dropna()
expected_diff = timedelta(minutes=30)
irregular_intervals = time_diffs[time_diffs != expected_diff]

if len(irregular_intervals) > 0:
    print("Irregular time intervals detected")
else:
    print("All intervals are 30 minutes")

All intervals are 30 minutes
